In [21]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
from bs4 import BeautifulSoup
import requests
import re

In [22]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [23]:
tokens = tokenizer.encode("this is good, but could be better", return_tensors='pt')

In [24]:
result = model(tokens)
result.logits

tensor([[-2.2128, -0.3438,  2.0223,  1.5271, -0.9159]],
       grad_fn=<AddmmBackward0>)

In [25]:
int(torch.argmax(result.logits)+1)


3

In [26]:
#collect reviews 
r = requests.get('https://de.trustpilot.com/review/www.saphe.com')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*typography_body-l.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]
 

In [27]:
reviews[9]

'Das Gerät an sich hat eine super Qualität und die App macht auch einen übersichtlichen soliden Eindruck . Man kann alles über die App steuern … wenn man an einem Blitzer vorbei fährt kann man diesen direkt am Gerät melden .Der Akku des Gerätes hält super lang (habe nach der Lieferung noch kein einziges Mal laden müssen . Habe das Gerät jetzt schon ungefähr eine Woche )Das einzige was mich stört , ich würde gerne wenn ein Blitzer kommt ein bisschen früher mit einem Ton gewarnt werden und nicht erst 300 Meter davor . Und irgendwie stimmt etwas mit der Kommunikation nicht was das Internet betrifft . Kann aber auch an mir liegen :)Im Großen und ganzen würde ich das Gerät weiter empfehlen und ich bin gespannt was die Zeit noch so mit sich bringen wird . '

In [28]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [29]:
df['review'].iloc[2]

'Die Kombination von Navigation und Radarwarnung funktioniert gut bis sehr gut. Die Signaltöne sind differenziert und ausreichend laut.Toll wäre die Möglichkeit eigene Töne zu definieren (z.B. "Du fährst zu schnell" mit Stimme der Ehefrau")Schwierig ist die Bedienung während der Fahrt, insbesondere die Start-Taste für die Navigation und die Zentrierfunktion sind viel zu klein.'

In [30]:
def sentiment_pred(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)+1)


In [31]:
sentiment_pred(df['review'].iloc[2])
df = df.drop(index=[0, 1, 22, 23, 24])

In [32]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_pred(x[:512]))

In [33]:
df

,review,sentiment
2,Die Kombination von Navigation und Radarwarnun...,4
3,Ich habe seit 1 Jahr das Saphe Drive Pro und b...,4
4,"Alles gut, alles funktioniert wie es soll. Bat...",5
5,Gut ist der Display der die Geschwindigkeit un...,4
6,Das Gerät hat bisher immer zuverlässig alle Bl...,5
7,Ich habe es getestet für den pkw und bin posit...,5
8,"Das Gerät an sich ist recht gut, Akku hält lan...",4
9,Das Gerät an sich hat eine super Qualität und ...,4
10,Super Gerät. Durch die Vernetzung werden sämtl...,4
11,Was einfach nervt an den Geräten ist das selbe...,1
